[Reference](https://medium.com/@ccpythonprogramming/building-a-notification-service-with-fastapi-and-python-474e877329f7)

# Setting Up the Project

```
notification_service/
├── main.py
├── models.py
├── database.py
├── notification.db
```

In [1]:
!pip install fastapi uvicorn sqlalchemy pydantic sqlite3

ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3


# Creating the Database Model

In [2]:
from sqlalchemy import Column, Integer, String, DateTime
from sqlalchemy.ext.declarative import declarative_base
from datetime import datetime

Base = declarative_base()

class Notification(Base):
    __tablename__ = "notifications"

    id = Column(Integer, primary_key=True, index=True)
    recipient = Column(String, index=True)
    message = Column(String)
    status = Column(String, default="pending")
    created_at = Column(DateTime, default=datetime.utcnow)

# Database Connection

In [3]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from models import Base

SQLALCHEMY_DATABASE_URL = "sqlite:///./notification.db"

engine = create_engine(SQLALCHEMY_DATABASE_URL, connect_args={"check_same_thread": False})
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

def init_db():
    Base.metadata.create_all(bind=engine)

# FastAPI Application

In [4]:
from fastapi import FastAPI, Depends
from sqlalchemy.orm import Session
from pydantic import BaseModel
from models import Notification
from database import SessionLocal, init_db

app = FastAPI()

# Initialize the database
init_db()

# Dependency for database session
def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()

# Pydantic model for request validation
class NotificationRequest(BaseModel):
    recipient: str
    message: str

@app.post("/notifications/")
async def create_notification(notification_req: NotificationRequest, db: Session = Depends(get_db)):
    notification = Notification(
        recipient=notification_req.recipient,
        message=notification_req.message,
        status="pending",
    )
    db.add(notification)
    db.commit()
    db.refresh(notification)
    return {"status": "Notification created", "notification_id": notification.id}

@app.get("/notifications/{notification_id}")
async def read_notification(notification_id: int, db: Session = Depends(get_db)):
    notification = db.query(Notification).filter(Notification.id == notification_id).first()
    if notification is None:
        return {"error": "Notification not found"}
    return {
        "id": notification.id,
        "recipient": notification.recipient,
        "message": notification.message,
        "status": notification.status,
        "created_at": notification.created_at,
    }

@app.put("/notifications/{notification_id}")
async def update_notification_status(notification_id: int, status: str, db: Session = Depends(get_db)):
    notification = db.query(Notification).filter(Notification.id == notification_id).first()
    if notification is None:
        return {"error": "Notification not found"}

    notification.status = status
    db.commit()
    return {"status": "Notification updated", "notification_id": notification.id}

# Running the Service

```
uvicorn main:app --reload
```

# Testing the API Endpoints

```
curl -X 'POST' \
  'http://127.0.0.1:8000/notifications/' \
  -H 'Content-Type: application/json' \
  -d '{
  "recipient": "john.doe@example.com",
  "message": "You have a new alert."
}'
```